In [208]:
import string
import random
import time
import math

import torch.nn as nn
import shapefile
from tqdm import tqdm_notebook as tqdm
import sklearn.feature_extraction
import numpy
import torch
import torch.random
import matplotlib.pyplot as plt

In [200]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inp, hidden):
        batch_size = inp.size(0)
        encoded = self.encoder(inp)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, inp, hidden):
        encoded = self.encoder(inp.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.autograd.Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                torch.autograd.Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))

In [10]:
suburb_shape = shapefile.Reader('../NSW_LOCALITY_POLYGON_shp/NSW_LOCALITY_POLYGON_shp')

In [14]:
names = [r.NSW_LOCA_2 for r in suburb_shape.records()]

In [100]:
letters = sorted(string.ascii_uppercase + "'- *")

In [270]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = letters.index(string[c])
        except:
            continue
    return tensor

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def random_training_set(corpus, chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, len(corpus) - chunk_len - 1)
        end_index = start_index + chunk_len + 1
        chunk = corpus[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = torch.autograd.Variable(inp)
    target = torch.autograd.Variable(target)
    return inp, target

def train(inp, target, batch_size, chunk_len, decoder, optimiser):
    hidden = decoder.init_hidden(batch_size)
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    optimiser.step()

    return loss.data[0] / chunk_len

In [271]:
def generate(decoder, prime_str='****', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden(1)
    prime_input = torch.autograd.Variable(char_tensor(prime_str).unsqueeze(0))

    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = letters[top_i]
        predicted += predicted_char
        inp = torch.autograd.Variable(char_tensor(predicted_char).unsqueeze(0))

    return predicted

In [272]:
# inputs = []
# outputs = []
# window_size = 4
# for name in tqdm(names):
#     name = '*' * window_size + name + '****'
#     for i in range(len(name) - window_size):
#         inputs.append(name[i:i + window_size])
#         outputs.append(name[i + window_size])

In [273]:
input_corpus = ''.join('*' * 4 + n for n in names) + '****'

In [274]:
# letter_to_int = {l:i for i, l in enumerate(letters)}

In [275]:
# input_arr = numpy.zeros((len(inputs), 4, len(letters)))
# for n, inp in enumerate(inputs):
#     for i, l in enumerate(inp):
#         input_arr[n, i, letter_to_int[l]] = 1

In [276]:
# output_arr = numpy.zeros((len(inputs), len(letters)))
# for n, out in enumerate(outputs):
#     output_arr[n, letter_to_int[l]] = 1

In [277]:
decoder = RNN(
    len(letters),
    16,
    len(letters),
    n_layers=1,
)

In [278]:
decoder_optimiser = torch.optim.Adam(decoder.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
all_losses = []

In [ ]:
start = time.time()
loss_avg = 0
n_epochs = 100000
batch_size = 128
chunk_len = 4
for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set(input_corpus, chunk_len, batch_size), batch_size, chunk_len,
                 decoder, decoder_optimiser)
    loss_avg += loss

    if epoch % 100 == 1:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(generate(decoder, '****', 100), '\n')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


 0s (1 0%) 1.8827]
****CAMBA***********************BRUND****************KESHILMTON****************************BOONDI******* 

 2s (101 0%) 1.7728]
****BAGE****************LEAD****************TANDEN HOLLE*************BARRIGH****************WAMBURRONG** 

 4s (201 0%) 1.7724]
****REEMBOIWARE****************THALLERIGLENAYANAH*********************WOOWALLEI************************* 

 5s (301 0%) 1.9111]
****COOUN BRIEM*****************BERREEK*************CAH HURNTS**************RUNT**********************LA 

 7s (401 0%) 1.9237]
****MOOT PARRABAIT*****************NINHAN******************THENG************NCHAVILL*****************ALL 

 10s (501 0%) 1.8637]
****BA BIONG*****************TACKILLE***************DEENNORS MOULBI**************GOODHON**************** 

 12s (601 0%) 1.8346]
****RROOREE************NTAH****************NERINGSY ROWONTHOO***************MARAB*******GRACH*********** 

 14s (701 0%) 1.7630]
****CONMIST***************MONG VALE************THORTH PALIVAN***************BARRA*

 7s (6201 6%) 1.7286]
****MUNDEN****************************************COOD************************************TOALMUNTH BEAR 

 9s (6301 6%) 1.7596]
****LASWAREE*******************BOBINT*************************DOONBEMPI NOGREEFT HILLE HILLE************ 

 10s (6401 6%) 1.9274]
****TALETER*****************WEACK**********************MIIGA******************MARRINER HILL************* 

 12s (6501 6%) 1.9390]
****RIGTH****************AMARRS GROO***********EANT****************EEM**********ROWN CREEK*******EBUBAL* 

 14s (6601 6%) 1.8166]
****CERAMBRA****************BIDMIOND PARKE****************KINTON****************MOURRIDGE**************T 

 17s (6701 6%) 1.8186]
****CATEEK************KIRROCKEON*****************LEANGLA*****************BOGHEY**************EAST CREEK* 

 19s (6801 6%) 1.8444]
****YENT CROST************ELILLS*************EBOMOOK****************EGRELLIG******************CREEK***** 

 21s (6901 6%) 1.9081]
****WOON****************EANT NANDA**************WINNARN************EL

 22s (12401 12%) 1.7649]
*****MIOD********LULLE ROVE***********EADVILLE**********EAWTON*********HILLE***********EAGHE POINT CREEK 

 25s (12501 12%) 1.9008]
****GREEBOW**********ELAMARK*********ELUNTER PARDON***********EEAVEN**********OYEN************EAGHATT HI 

 27s (12601 12%) 1.8004]
****FREEK**********HAID*******FALLGERGALA************EACDALE***********EATHANGI**********ALEEK*********E 

 28s (12701 12%) 1.8620]
****MUGLE GABABROS*********ELADDI CREEK********EATHAMBALONVALETHER*********EOVE***********EIRDOY******** 

 30s (12801 12%) 1.7894]
****WEELGRA************EIEENANG HILLE************EDUDGER POINGAND************ALEDIS CREEK*********EUCK** 

 32s (12901 12%) 2.0077]
****EUTTIS PIRONDS***********EANGETS PARKTARRI********EWEEDDNERA************EARST CREED******SPRIND***** 

 34s (13001 13%) 1.7184]
****FRSTAINT CREELDUN***********EENTEN NOTE**********EASTTANGLA************EIRAPPORI***********EESTH HIL 

 36s (13101 13%) 1.8445]
****CELADDARRAY*********EDEEN*********STAINE PRING*

 42s (18501 18%) 1.7685]
****PILDA*************EWOUDS CREEK************MIONTH ERAVE***********ELAVEAHOWGRESTEN*************EAREEK 

 48s (18601 18%) 1.8212]
****MOUNTWAMBULLI POINT ALE***********EACK************EAGHEAVILLY***********ASVEISF PLAREEK**********EAS 

 50s (18701 18%) 1.8323]
****INDELLS*************OONITS*************YBOPPARK************ELIONTIE**********EITHEFLEMBILD MING MING 

 52s (18801 18%) 1.7956]
****ABK************EATHS*********EACDSORTON***********YOUGALONS BALLE CREEK*********AILDE**********EURID 

 55s (18901 18%) 1.7755]
****MOUNT*************OIDA************EUDEEDIANGLEALAWORTH**************MARRA**********YOOCDANGALA****** 

 59s (19001 19%) 1.8599]
****BARVERWELLIMBUL*************EPPEAKE***********EIGUNCH***********EAGHE*********EUILLLADE**********EAD 

 2s (19101 19%) 1.8564]
****WILDIE***********EINTH*************LIA**************EATTH**************RUOLINCHALLANADAND ROYS BANGI 

 5s (19201 19%) 1.8014]
****WARTHTON**********ERAMA**********AAKER********ELA

...

 7s (67301 67%) 1.8315]
****LELLE*********************************************************************************************** 

 10s (67401 67%) 1.6982]
****MPUWING********************************************************************************************* 

 12s (67501 67%) 1.7250]
****BANDACORA******************************************************************************************* 

 14s (67601 67%) 1.8654]
****BELLANVILLE****************************************************************************UPPEEENE WEST 

 16s (67701 67%) 1.7970]
****STY LIRARANG**************************************************************************************** 

 19s (67801 67%) 1.8442]
****COOMBTLEN****************************************************************EDARTWELDS***************** 

 21s (67901 67%) 1.8304]
****MUNTHA***************************************************************CAPPING************************ 

 23s (68001 68%) 1.9145]
****BLARINVE****************************************

 13s (73401 73%) 1.8860]
****RIGHE PLEAST CREEN********************************************************************************** 

 15s (73501 73%) 1.8043]
****NILLADA********************************************************************************************* 

 17s (73601 73%) 1.9129]
****GOCD************************************************************************************************ 

 19s (73701 73%) 1.8304]
****GAPSIN********************************************************************************************** 

 21s (73801 73%) 1.7065]
****COSTONES ALASTEK************************************************************************************ 

 22s (73901 73%) 1.8291]
****THER CROST****************************************************************************************** 

 24s (74001 74%) 1.8032]
****COALGIMYRICKENLE PANHIN***************************************************************************** 

 26s (74101 74%) 1.9248]
****COOLLANVINE************************************

 18s (79501 79%) 1.7918]
****DOLSHACK******************************************************************************************** 

 20s (79601 79%) 1.7984]
*****DUNGERRAYS RIVER****************************************************ELYDELDONS FLAREE************** 

 23s (79701 79%) 1.7260]
****HAVE************************************************************************************************ 

 25s (79801 79%) 1.8434]
****SQUIED POINT**************************************************************************************** 

 27s (79901 79%) 1.8271]
****WOLESP********************************************************************************************** 

 29s (80001 80%) 1.8064]
****COOM************************************************************************************************ 

 31s (80101 80%) 1.8953]
****WAKEE FOREER**************************************************************************************** 

 33s (80201 80%) 1.6503]
****MUNGHAPCUNNOW**********************************

 4s (85601 85%) 1.8885]
****MOUNTER CREEK*************************************************************************************** 

 7s (85701 85%) 1.6797]
****COE BEARTON***************************************************************************************** 

 10s (85801 85%) 1.8272]
****MOURRAWOONT SOUTTLE********************************************************************************* 

 13s (85901 85%) 1.7766]
****BELLAND RIGLEBACH*********************************************************************************** 

 16s (86001 86%) 1.7070]
****WARCH*********************************************************************************************** 

 18s (86101 86%) 1.6675]
****BEMBO*********************************************************************************************** 

 21s (86201 86%) 1.7704]
****BAREEK********************************************************************************************** 

 23s (86301 86%) 1.7677]
****RACHEET******************************************

 49s (91701 91%) 1.7820]
****WEWANS PARKELONG************************************************************************************ 

 52s (91801 91%) 1.9033]
****BRYDING********************************************************************************************* 

 56s (91901 91%) 1.7742]
****ROGBELL********************************************************************************************* 

 1s (92001 92%) 1.8820]
****NOME************************************************************************************************ 

 4s (92101 92%) 1.8010]
****NAPINMESTABUGARONG********************************************************************************** 

 7s (92201 92%) 1.8843]
****DALLAND********************************************************************************************* 

 10s (92301 92%) 1.7830]
****BURINDED******************************************************************************************** 

 13s (92401 92%) 1.9198]
****HLAND*********************************************

In [280]:
import pdb

In [ ]:
pdb.pm()

In [281]:
for i in range(10):
    print(generate(decoder, '****', 100), '\n')

****WEEST HELL***********************UPPERS RODFA********************DARALNANUNS HEACH**************KAR  

****UPPPER MART************************GOOK***************************WELKONS WALEN CREEK*********KENBTO 

****BOANGINGAROTRARRESELDALTONN*******************GURARRULD*******************PELWURRA**************MER  

****BA*******************TAREEK************MPEY***************************MOINT************************* 

****WEENTAN****************************NUNT BRI**********BATHALBILLUNTIN****************************MA** 

****GOUTHTS*******************TARRAGO**********************COOPBA*********************KENS PARK********* 

****MOOD*****************KELE HELS MOOMERT***************************TANTAH*********************MAMBAN** 

****SWOUO*****************************MOANT SOOREW BAREELL*************************COREEN*************** 

****MARTH VALLOO****************************TINVA*************WHYS***********REEK************THUNDMOBRIN 

****COOOD**********************CAT SI